In [1]:
%matplotlib inline
import zipline
from scipy import stats
from zipline import TradingAlgorithm
from zipline.api import order_target, record, symbol, history, add_history

# Returns the linear regression predicted value for the given 
# stock N days in advance for the data field, and period.
def get_expected(data_field, period, stock, days_advanced):
    data = history(bar_count=period, frequency="1d", field=data_field)[stock]
    day = len(data)
    days = list(range(day))
    slope, intercept, r_value, p_value, std_err = stats.linregress(days, data)    
    return slope * (day + days_advanced) + intercept


# Returns the slope of the linear regression for
# the specified data of the given period and security.
def get_slope(data_field, period, stock):   
    data = history(bar_count=period, frequency="1d", field=data_field)[stock]
    days = list(range(len(data)))
    slope, intercept, r_value, p_value, std_err = stats.linregress(days, data)
    return slope


# Put any initialization logic here.  The context object will be passed to
# the other methods in your algorithm.
def initialize(context):
    context.stock = symbol('SPY')
    set_benchmark(context.stock)


# Will be called on every trade event for the securities you specify. 
def handle_data(context, data):
    
    if get_open_orders(context.stock):
        return
    
    PERIOD = 70  # Number of days over which to make expectations.
    TRADING_YEAR = 252
    
    stock = context.stock
    stock_data = data[stock]
    
    expected_close = get_expected("close_price", PERIOD, stock, 1)
    expected_open = get_expected("open_price", PERIOD, stock, 1)
    expected_high = get_expected("high", PERIOD, stock, 1)
    expected_low = get_expected("low", PERIOD, stock, 1)       
    
    current_close = stock_data.close_price
    current_open = stock_data.open_price
    current_low = stock_data.low
    current_high = stock_data.high
    
    short_sdev = stock_data.stddev(int(PERIOD / 2))
    long_sdev = stock_data.stddev(PERIOD)
    
    volume_confirmation = get_slope("volume", PERIOD, stock) > 0
    
    market_type_slope = get_slope("close_price", TRADING_YEAR, stock)
    bull = market_type_slope > 0
    bear = market_type_slope < 0
    
    expectations = [expected_close > current_close,
                    expected_open > current_open,
                    expected_high > current_high,
                    expected_low > current_low]    
    
    # Returns the majority result
    probably_buy = max(set(expectations), key=expectations.count)   
    
    if bull:
        if probably_buy:
            order_target_percent(stock, 1)        
    elif bear:
        if probably_buy:
            if volume_confirmation:
                if short_sdev < long_sdev:
                    order_target_percent(stock, 1)
                else:
                    order_target_percent(stock, 0)
        else:
            order_target_percent(stock, 0)    

# Instantinate algorithm        
algo = TradingAlgorithm(initialize=initialize, 
                        handle_data=handle_data)
# Run algorithm
results = algo.run(data)
results.portfolio_value.plot()

ImportError: No module named 'zipline'